In [1]:
import numpy as np
from linear import LinintGrow, LinintEqui
from DiscretizeTools import grow_grid, rouwenhorst
from scipy.optimize import root



In [2]:
T = 40
J = 12
JR = 10
NP = 2
NS = 5
NA = 100

γ = 0.5
egam = 1 - 1/γ
ν = 0.335
β = 0.998**5

σθ = 0.23
σϵ = 0.05
ρ = 0.98

α = 0.36
δ = 1 - (1 - 0.0823)**5
Ω = 1.60

al = 0
au = 35
agrow = 0.05

n_p = (1 + 0.01)**5 -1
itermax = 50

damp = 0.3
sig = 1e-4



In [3]:
r = np.zeros(T+1)
rn = np.zeros(T+1)
w = np.zeros(T+1)
wn = np.zeros(T+1)
pp = np.zeros(T+1)

In [4]:
pop = np.zeros((J, T+1))
m = np.zeros((J, T+1))
aplus = np.zeros((J, NA + 1, NP, NS, T + 1))
c = np.zeros((J, NA + 1, NP, NS, T + 1))
l = np.zeros((J, NA + 1, NP, NS, T + 1))
VV = np.zeros((J, NA + 1, NP, NS, T + 1))
v = np.zeros((J, NA + 1, NP, NS, T + 1))
RHS = np.zeros((J, NA + 1, NP, NS, T + 1))
EV = np.zeros((J, NA + 1, NP, NS, T + 1))
Φ = np.zeros((J, NA + 1, NP, NS, T + 1))


c_coh = np.zeros((J,T+1))
l_coh = np.zeros((J,T+1))
y_coh = np.zeros((J,T+1))
a_coh = np.zeros((J,T+1))
v_coh = np.zeros((J,T+1))
VV_coh = np.zeros((J,T+1))
FLC = np.zeros((J,T+1))
m = np.zeros((J,T+1))
PP = np.zeros(T+1)

taxrev = np.zeros((4,T+1))

eff = np.zeros(J)

distθ = [0.5,0.5]

θ = np.exp(np.array([-σθ**0.5,σθ**0.5]))

η, π = rouwenhorst(ρ, σϵ, NS)
η = np.exp(η)


for j in range(J):
    pop[j, 0] = 1/(1+n_p)**j
    
for j in range(J):
    m[j, 0] = pop[j,0]/pop[0,0]

a = grow_grid(al, au, agrow, NA+1)


for j in range(J):
    for p in range(NP):
        for s in range(NS):
            aplus[j, :, p, s, 0] = np.maximum(a/2, np.ones(NA+1)*a[1]/2)
print(a/2)
# print(aplus[10,:,0,0,0])

eff[0] = 1.0000
eff[1] = 1.3527
eff[2] = 1.6952
eff[3] = 1.8279
eff[4] = 1.9606
eff[5] = 1.9692
eff[6] = 1.9692
eff[7] = 1.9392
eff[8] = 1.9007
eff[JR-1:] = 0 

tax = np.ones(T+1)*2
τc = np.ones(T+1)*0.075
τw = np.zeros(T+1)
τr = np.zeros(T+1)
τp = np.ones(T+1)*0.1
κ = np.ones(T+1)*0.5

gy = 0.19
by = 0.6/5

K = np.ones(T+1) 
L = np.ones(T+1)
Y = np.ones(T+1)
C = np.ones(T+1)
H = np.ones(T+1)
A = np.ones(T+1)
BA = np.ones(T+1)
I = (n_p+δ)*K
INC = np.ones(T+1)
DIFF = np.ones(T+1)

G = np.ones(T+1)*gy*Y[0]
B = np.ones(T+1)*by*Y[0]

pen = np.zeros((J,T+1))
pen[JR-1:,0] = κ[0]
    


[0.00000000e+00 6.70491622e-03 1.37450782e-02 2.11372484e-02
 2.88990270e-02 3.70488946e-02 4.56062555e-02 5.45914845e-02
 6.40259750e-02 7.39321899e-02 8.43337156e-02 9.52553176e-02
 1.06723000e-01 1.18764066e-01 1.31407185e-01 1.44682461e-01
 1.58621500e-01 1.73257491e-01 1.88625282e-01 2.04761463e-01
 2.21704452e-01 2.39494591e-01 2.58174236e-01 2.77787864e-01
 2.98382174e-01 3.20006199e-01 3.42711425e-01 3.66551912e-01
 3.91584424e-01 4.17868562e-01 4.45466906e-01 4.74445168e-01
 5.04872342e-01 5.36820875e-01 5.70366835e-01 6.05590093e-01
 6.42574514e-01 6.81408156e-01 7.22183480e-01 7.64997571e-01
 8.09952365e-01 8.57154900e-01 9.06717561e-01 9.58758355e-01
 1.01340119e+00 1.07077616e+00 1.13101989e+00 1.19427580e+00
 1.26069451e+00 1.33043415e+00 1.40366077e+00 1.48054873e+00
 1.56128108e+00 1.64605005e+00 1.73505747e+00 1.82851526e+00
 1.92664594e+00 2.02968315e+00 2.13787222e+00 2.25147075e+00
 2.37074920e+00 2.49599158e+00 2.62749608e+00 2.76557580e+00
 2.91055950e+00 3.062792

In [5]:
# current time is t, calculate at which year the agent with age j turns to be age p
def year(t,j,p):
    year = t + p -j
    
    if t == 0 or year <= 0 : year = 0
        
    if t == T or year >= T : year =  T
    
    return year

In [6]:
def margu(cons,lab,it):
    return ν*(cons**ν*(1-lab)**(1-ν))**(1-1/γ)/(pp[it]*cons)


def interpolate(j,t):
    for i in range(NA+1):
        for p in range(NP):
            for s in range(NS):
                RHS[j,i,p,s,t] = 0
                EV[j,i,p,s,t] = 0
                
                for sp in range(NS):
                    chelp = max(c[j,i,p,sp,t],1e-10)
                    lhelp = max(l[j,i,p,sp,t],1e-10)
                    RHS[j,i,p,s,t] += π[s,sp]*margu(chelp,lhelp,t)
                    EV[j,i,p,s,t]  += π[s,sp]*VV[j,i,p,sp,t]
                RHS[j,i,p,s,t] = ((1+rn[t])*β*RHS[j,i,p,s,t])**(-γ)
                EV[j,i,p,s,t] = ((1-1/γ)*EV[j,i,p,s,t])**(1/(1-1/γ))
    return None
    
    

In [7]:
def get_distribution(t):
    tm = year(t,1,0)
    
    Φ[:,:,:,:,t] = 0
    
    for p in range(NP):
        Φ[0,0,p,2,t] = distθ[p]
        
    for j in range(1,J):
        for i in range(NA+1):
            for p in range(NP):
                for s in range(NS):
                    
                    ial, iar, φ = LinintGrow(aplus[j-1,i,p,s,tm], al, au, agrow, NA+1)
                    
                    ial = max(min(ial, NA-1),0)
                    iar = max(min(iar, NA)  ,1)
                    φ   = max(min(φ  , 1 )  ,0)
                    
                    for sp in range(NS):
                        Φ[j,ial,p,sp,t] += π[s,sp]*φ    *Φ[j-1,i,p,s,tm]
                        Φ[j,iar,p,sp,t] += π[s,sp]*(1-φ)*Φ[j-1,i,p,s,tm]
    
    
    return None
    

In [8]:
def aggregation(t):
    
    tp = year(t,0,1)
    
    Lold = L[t]
    
    c_coh[:,t] = 0
    l_coh[:,t] = 0
    y_coh[:,t] = 0
    a_coh[:,t] = 0
    VV_coh[:,t] = 0
    FLC[:,t] = 0
    
    m_coh = np.zeros(J)
    
    for j in range(J):
        for i in range(NA+1):
            for p in range(NP):
                for s in range(NS):
                    
                    c_coh[j,t] += c[j,i,p,s,t]*Φ[j,i,p,s,t]
                    l_coh[j,t] += l[j,i,p,s,t]*Φ[j,i,p,s,t]
                    y_coh[j,t] += eff[j]*θ[p]*η[s]*l[j,i,p,s,t]*Φ[j,i,p,s,t]
                    a_coh[j,t] += a[i]*Φ[j,i,p,s,t]
                    
                    if j>=JR-1 and i == 0 and (κ[0]<= 1e-10 or κ[1]<=1e-10):
                        continue
                    if aplus[j,i,p,s,t] < 1e-4:
                        FLC[j,t] += Φ[j,i,p,s,t]
                    VV_coh[j,t] += VV[j,i,p,s,t]*Φ[j,i,p,s,t]
                    m_coh[j] += Φ[j,i,p,s,t]

    
    VV_coh[:,t] = VV_coh[:,t]/m_coh
    FLC[:,t] = FLC[:,t]/m_coh
    
    
    
    C[t] = 0
    L[t] = 0
    H[t] = 0
    A[t] = 0
    workpop = 0
    
    for j in range(J):
        C[t] += c_coh[j,t]*m[j,t]
        L[t] += y_coh[j,t]*m[j,t]
        H[t] += l_coh[j,t]*m[j,t]
        A[t] += a_coh[j,t]*m[j,t]
        
        if j < JR-1:
            workpop += m[j,t]
    
    
    K[t] = damp*(A[t] - B[t] - BA[t]) + (1-damp)*K[t]
    L[t] = damp*L[t] + (1-damp)*Lold
    I[t] = (1+n_p)*K[tp] - (1-δ)*K[t]
    Y[t] = Ω * K[t]**α *L[t]**(1-α)
    
    INC[t] = w[t]*L[t]/workpop
    H[t] = H[t]/workpop
    
    DIFF[t] = Y[t] -C[t]- I[t] -G[t]
    
    return None
                        



In [9]:
def government(t):
    tm = year(t,1,0)
    tp = year(t,0,1)
    
    G[t] = gy*Y[0]
    B[t] = by*Y[0]
    pen[JR-1:, t] = κ[t]*INC[tm]
    PP[t] = 0
    
    for j in range(JR-1,J):
        PP[t] += pen[j,t]*m[j,t]
        
    expand = G[t] + (1+r[t])*B[t] - (1+n_p)*B[tp]
    
    if tax[t] == 1:
        τc[t] = (expand - (τw[t]*w[t]*L[t] + τr[t]*r[t]*A[t]))/C[t]
        pp[t] = 1 + τc[t]
    
    elif tax[t] == 2:
        τw[t] = (expand - τc[t]*C[t])/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]
    elif tax[t] == 3:
        τw[t] = (expand - (τc[t]*C[t] + τr[t]*r[t]*A[t]))/(w[t]*L[t])
    else:
        τr[t] = (expand - (τc[t]*C[t] + τw[t]*w[t]*L[t]))/(r[t]*A[t])
        
    
    taxrev[0,t] = τc[t]*C[t]
    taxrev[1,t] = τw[t]*w[t]*L[t]
    taxrev[2,t] = τr[t]*r[t]*A[t]
    taxrev[3,t] = sum(taxrev[:3,t])
    
    τp[t] = PP[t]/(w[t]*L[t])
    
    return None









In [10]:
def prices(t):    
    r[t] = Ω*α*(K[t]/L[t])**(α-1)-δ
    w[t] = Ω*(1-α)*(K[t]/L[t])**α
    rn[t] = r[t]*(1-τr[t])
    wn[t] = w[t]*(1-τw[t]-τp[t])
    pp[t] = 1+τc[t]
    return None


In [11]:
def valuefunc(ap, cons, lab, j, p, s, t):
    chelp = max(cons,1e-10)
    lhelp = min(max(lab,0),1-1e-10)
    
    tp = year(t,j,j+1)
    
    ial, iar, ϕ = LinintGrow(ap,al,au,agrow,NA+1)
    
    val = 0
    
    if j < J-1:
        val = max(ϕ*EV[j+1, ial, p, s, tp] + (1-ϕ)*EV[j+1, iar, p, s, tp], 1e-10)**(1-1/γ)/(1-1/γ)

    val = (chelp**ν*(1-lhelp)**(1-ν))**(1-1/γ)/(1-1/γ) + β*val
    return val
    
    
    
    

In [12]:
def foc(xin,i, j, p, s, t):
    ap = xin
    tp = year(t,j,j+1)
    vind = v[j, i, p, s, t]
    
    wage = wn[t]*eff[j]*θ[p]*η[s]
    
    available = (1 + rn[t])*a[i] + pen[j,t] + vind
    
    if j < JR-1:
        labcom = min(max(ν + (1-ν)*(ap-available)/wage, 0),1-1e-10)
    else:
        labcom = 0
        
    conscom = max((available + wage*labcom -ap)/pp[t],1e-10) 
    
    ial,iar,φ = LinintGrow(ap, al, au, agrow, NA+1)
#     print(ap,al,au,agrow)
#     print(ial,iar,φ)
    
    tomorrow = max(φ*RHS[j+1,ial,p,s,tp]+(1-φ)*RHS[j+1, iar,p,s,tp],0)
    foc = margu(conscom, labcom, t)**(-γ) - tomorrow
    
    return foc
        
        

In [13]:
def solve_household(j, t):
    it = year(t, j, J-1)
    
    for i in range(NA+1):
        aplus[J-1,i,:,:,it] = 0
        c[J-1,i,:,:,it] = ((1+rn[it])*a[i]+ pen[J-1,it] + v[J-1,i,:,:,it])/pp[it]
        l[J-1,i,:,:,it] = 0
        VV[J-1,i,:,:,it] = valuefunc(0, c[J-1, i, 0, 0, it], l[J-1, i, 0, 0, it], J-1, 0,0, it)
    
    interpolate(J-1, it)

    for ij in range(J-2, j-1, -1):
        
        it = year(t, j, ij)
        
        if ij >= JR-1:
            ipmax = 1
            ismax = 1
        else:
            ipmax = NP
            ismax = NS
            
        for i in range(NA+1):
            if (ij >= JR-1 and i == 0 and κ[t] <= 1e-10):
                aplus[ij,i,:,:,it] = 0
                c[ij,i,:,:,it] = 0
                l[ij,i,:,:,it] = 0
                VV[ij,i,:,:,it] = valuefunc(0,0,0,ij,0,0,it)
                continue
            
            for p in range(ipmax):
                for s in range(ismax):
                    print(ij,i,p,s,it)                    
                    
                    res = root(foc, x0 = aplus[ij,i,p,s,it], args = (i, ij, p, s, it), tol = 1e-8)
                    
                    if res.x[0] < 0:
                        aplus[ij,i,p,s,it] = 0
                        wage = wn[it]*eff[ij]*θ[p]*η[s]
                        vind = v[ij,i,p,s,it]
                        available = (1+rn[it])*a[i] + pen[ij,it] + vind
                        if ij < JR-1:
                            labcom = min(max(ν-(1-ν)*available/wage,0), 1-1e-10)
                        else:
                            labcom = 0
                        conscom = max((available + wage *labcom)/pp[it],1e-10)
                    else:
                        aplus[ij,i,p,s,it] = res.x[0]
                        vind = v[ij, i, p, s, t]
                        wage = wn[t]*eff[ij]*θ[p]*η[s]
                        available = (1 + rn[t])*a[i] + pen[ij,t] + vind
                        if ij < JR-1:
                            labcom = min(max(ν + (1-ν)*(aplus[ij,i,p,s,it]-available)/wage, 0),1-1e-10)
                        else:
                            labcom = 0
                        conscom = max((available + wage*labcom -aplus[ij,i,p,s,it])/pp[t],1e-10) 
                    
                    
                    

                    c[ij,i,p,s,it] = conscom
                    l[ij,i,p,s,it] = labcom
                    VV[ij,i,p,s,it] = valuefunc(res.x[0], conscom, labcom, ij, p, s, it)
#                     print('ap',aplus[ij,i,p,s,it])
#                     print('c',c[ij,i,p,s,it])
#                     print('l',l[ij,i,p,s,it])
                    print('VV',VV[ij,i,p,s,it])
                    
                
                if ij >= JR - 1:
                    aplus[ij,i,:,:,it] = aplus[ij,i,0,0,it]
                    c[ij,i,:,:,it] = c[ij,i,0,0,it]
                    l[ij,i,:,:,it] = l[ij,i,0,0,it]
                    VV[ij,i,:,:,it] = VV[ij,i,0,0,it]
            
        interpolate(ij,it)
#         if ij == 10:
#             print('RHS',RHS.sum())
#             print('EV',EV.sum())
        
    return None
    
    
    
    

In [14]:
def get_SteadyState():
    
    for ite in range(itermax):
        prices(0)
        solve_household(0,0)
#         print('aplus=',aplus.sum())
#         print('c=',c.sum())
#         print('l=',l.sum())
#         print('VV=',VV.sum())
        get_distribution(0)
        aggregation(0)
        government(0)
        
        print(ite, H[0], DIFF[0]/Y[0])
        
        if abs(DIFF[0]/Y[0])*100 < sig:
            break
    return None
        
        

    

In [15]:
get_SteadyState()

10 0 0 0 0
VV -2.5687444084145694
10 1 0 0 0
VV -2.5533075068272417
10 2 0 0 0
VV -2.5375386411511656
10 3 0 0 0
VV -2.521328555624587
10 4 0 0 0
VV -2.504816136146163
10 5 0 0 0
VV -2.487881156748891
10 6 0 0 0
VV -2.47060126171591
10 7 0 0 0
VV -2.4529659254085834
10 8 0 0 0
VV -2.4349284850834385
10 9 0 0 0
VV -2.4166224123831137
10 10 0 0 0
VV -2.39793387394339
10 11 0 0 0
VV -2.3789000961031617
10 12 0 0 0
VV -2.3595982888076876
10 13 0 0 0
VV -2.3399493899350405
10 14 0 0 0
VV -2.3199844310044546
10 15 0 0 0
VV -2.299789947562716
10 16 0 0 0
VV -2.279288905896136
10 17 0 0 0
VV -2.2584961212073713
10 18 0 0 0
VV -2.2375012428178414
10 19 0 0 0
VV -2.216268613606735
10 20 0 0 0
VV -2.194789867375543
10 21 0 0 0
VV -2.1730813130497406
10 22 0 0 0
VV -2.1512357636213792
10 23 0 0 0
VV -2.129202627362288
10 24 0 0 0
VV -2.1069879011381225
10 25 0 0 0
VV -2.084608463764189
10 26 0 0 0
VV -2.062107027799632
10 27 0 0 0
VV -2.0395219252182315
10 28 0 0 0
VV -2.0168204728431496
10 29 0 0

8 6 1 4 0
VV -4.420758968086657
8 7 0 0 0
VV -5.866791325854692
8 7 0 1 0
VV -5.703290902642254
8 7 0 2 0
VV -5.507256803025141
8 7 0 3 0
VV -5.362436215909438
8 7 0 4 0
VV -5.221027160319231
8 7 1 0 0
VV -5.2495831150205685
8 7 1 1 0
VV -5.079323877547861
8 7 1 2 0
VV -4.878465448353916
8 7 1 3 0
VV -4.653435992087285
8 7 1 4 0
VV -4.411300719733484
8 8 0 0 0
VV -5.795103505539648
8 8 0 1 0
VV -5.644789599727554
8 8 0 2 0
VV -5.4599403403365905
8 8 0 3 0
VV -5.336187823821348
8 8 0 4 0
VV -5.198676714458809
8 8 1 0 0
VV -5.226499412872686
8 8 1 1 0
VV -5.060029879505633
8 8 1 2 0
VV -4.862683272390026
8 8 1 3 0
VV -4.640818669872655
8 8 1 4 0
VV -4.401418345127888
8 9 0 0 0
VV -5.7231626336792525
8 9 0 1 0
VV -5.585522896530608
8 9 0 2 0
VV -5.4116341104118995
8 9 0 3 0
VV -5.309030614521976
8 9 0 4 0
VV -5.175446818248481
8 9 1 0 0
VV -5.20259113669228
8 9 1 1 0
VV -5.040006335136304
8 9 1 2 0
VV -4.846354521511946
8 9 1 3 0
VV -4.627744229945076
8 9 1 4 0
VV -4.391095189961262
8 10 

VV -3.233479156110623
8 56 1 4 0
VV -3.2097335415260924
8 57 0 0 0
VV -3.1910881218995666
8 57 0 1 0
VV -3.1910881218995666
8 57 0 2 0
VV -3.1910881218995666
8 57 0 3 0
VV -3.1910881218995666
8 57 0 4 0
VV -3.1910881218995666
8 57 1 0 0
VV -3.1910881218995666
8 57 1 1 0
VV -3.1910881218995666
8 57 1 2 0
VV -3.1910881218995666
8 57 1 3 0
VV -3.1889282040129974
8 57 1 4 0
VV -3.1693461188264815
8 58 0 0 0
VV -3.1450811635252127
8 58 0 1 0
VV -3.1450811635252127
8 58 0 2 0
VV -3.1450811635252123
8 58 0 3 0
VV -3.1450811635252127
8 58 0 4 0
VV -3.1450811635252127
8 58 1 0 0
VV -3.1450811635252127
8 58 1 1 0
VV -3.1450811635252127
8 58 1 2 0
VV -3.1450811635252123
8 58 1 3 0
VV -3.144256439480849
8 58 1 4 0
VV -3.1285246544959127
8 59 0 0 0
VV -3.0994946554713603
8 59 0 1 0
VV -3.09949465547136
8 59 0 2 0
VV -3.0994946554713603
8 59 0 3 0
VV -3.0994946554713603
8 59 0 4 0
VV -3.0994946554713603
8 59 1 0 0
VV -3.0994946554713603
8 59 1 1 0
VV -3.09949465547136
8 59 1 2 0
VV -3.09949465547136

8 85 0 4 0
VV -2.080082310852604
8 85 1 0 0
VV -2.080082310852604
8 85 1 1 0
VV -2.080082310852604
8 85 1 2 0
VV -2.080082310852604
8 85 1 3 0
VV -2.080082310852604
8 85 1 4 0
VV -2.080082310852604
8 86 0 0 0
VV -2.0473996959145575
8 86 0 1 0
VV -2.0473996959145575
8 86 0 2 0
VV -2.0473996959145575
8 86 0 3 0
VV -2.0473996959145575
8 86 0 4 0
VV -2.0473996959145575
8 86 1 0 0
VV -2.0473996959145575
8 86 1 1 0
VV -2.0473996959145575
8 86 1 2 0
VV -2.0473996959145575
8 86 1 3 0
VV -2.0473996959145575
8 86 1 4 0
VV -2.0473996959145575
8 87 0 0 0
VV -2.01518302805788
8 87 0 1 0
VV -2.01518302805788
8 87 0 2 0
VV -2.01518302805788
8 87 0 3 0
VV -2.01518302805788
8 87 0 4 0
VV -2.01518302805788
8 87 1 0 0
VV -2.01518302805788
8 87 1 1 0
VV -2.01518302805788
8 87 1 2 0
VV -2.01518302805788
8 87 1 3 0
VV -2.01518302805788
8 87 1 4 0
VV -2.01518302805788
8 88 0 0 0
VV -1.9834286090028856
8 88 0 1 0
VV -1.9834286090028859
8 88 0 2 0
VV -1.9834286090028859
8 88 0 3 0
VV -1.9834286090028859
8 88 0

VV -6.5728336541032055
7 12 1 0 0
VV -6.615127685571085
7 12 1 1 0
VV -6.323476916492568
7 12 1 2 0
VV -6.004613046975665
7 12 1 3 0
VV -5.668889306992714
7 12 1 4 0
VV -5.325810835767965
7 13 0 0 0
VV -7.461760318033368
7 13 0 1 0
VV -7.241737537279098
7 13 0 2 0
VV -7.003548947018783
7 13 0 3 0
VV -6.79671596800499
7 13 0 4 0
VV -6.543642382790262
7 13 1 0 0
VV -6.584954675506738
7 13 1 1 0
VV -6.299484712259858
7 13 1 2 0
VV -5.985825958755937
7 13 1 3 0
VV -5.65450631594984
7 13 1 4 0
VV -5.314842967405448
7 14 0 0 0
VV -7.372373379221406
7 14 0 1 0
VV -7.17236237546484
7 14 0 2 0
VV -6.958245247918116
7 14 0 3 0
VV -6.7594483437263895
7 14 0 4 0
VV -6.513386465512882
7 14 1 0 0
VV -6.5536788514359525
7 14 1 1 0
VV -6.274542088597796
7 14 1 2 0
VV -5.966346920084256
7 14 1 3 0
VV -5.639485620925884
7 14 1 4 0
VV -5.303451127859365
7 15 0 0 0
VV -7.281654269529216
7 15 0 1 0
VV -7.101648700785239
7 15 0 2 0
VV -6.911524698419194
7 15 0 3 0
VV -6.720901526933072
7 15 0 4 0
VV -6.4819

...

6 35 1 3 0
VV -6.779697699806445
6 35 1 4 0
VV -6.428582944366132
6 36 0 0 0
VV -7.8034080523017355
6 36 0 1 0
VV -7.802576523925452
6 36 0 2 0
VV -7.77201910741338
6 36 0 3 0
VV -7.6781449445472
6 36 0 4 0
VV -7.514811914611579
6 36 1 0 0
VV -7.539761033073448
6 36 1 1 0
VV -7.321117119215891
6 36 1 2 0
VV -7.049461551824865
6 36 1 3 0
VV -6.736673339053799
6 36 1 4 0
VV -6.3951912755960905
6 37 0 0 0
VV -7.680797815628199
6 37 0 1 0
VV -7.680654094877631
6 37 0 2 0
VV -7.659823950179046
6 37 0 3 0
VV -7.580525780452286
6 37 0 4 0
VV -7.432323346861583
6 37 1 0 0
VV -7.455579526810407
6 37 1 1 0
VV -7.251548181028731
6 37 1 2 0
VV -6.993253914648793
6 37 1 3 0
VV -6.692177972753747
6 37 1 4 0
VV -6.360554555768373
6 38 0 0 0
VV -7.559996004454414
6 38 0 1 0
VV -7.559931589101526
6 38 0 2 0
VV -7.546973787554904
6 38 0 3 0
VV -7.481461283579764
6 38 0 4 0
VV -7.3482867521979855
6 38 1 0 0
VV -7.3697665310041955
6 38 1 1 0
VV -7.180391481242614
6 38 1 2 0
VV -6.9353756409943035
6 38 1 3

VV -4.8192788192531015
6 66 0 3 0
VV -4.8192788192531015
6 66 0 4 0
VV -4.8192788192531015
6 66 1 0 0
VV -4.819278819073999
6 66 1 1 0
VV -4.819278807465764
6 66 1 2 0
VV -4.819277946569883
6 66 1 3 0
VV -4.818155683317443
6 66 1 4 0
VV -4.797100567054794
6 67 0 0 0
VV -4.741727128200639
6 67 0 1 0
VV -4.741727128200639
6 67 0 2 0
VV -4.741727128200638
6 67 0 3 0
VV -4.741727128200638
6 67 0 4 0
VV -4.741727128200639
6 67 1 0 0
VV -4.7417271281445315
6 67 1 1 0
VV -4.741727123725849
6 67 1 2 0
VV -4.741726738654393
6 67 1 3 0
VV -4.7415450916531405
6 67 1 4 0
VV -4.7267383808391426
6 68 0 0 0
VV -4.665394230640648
6 68 0 1 0
VV -4.665394230640648
6 68 0 2 0
VV -4.665394230640648
6 68 0 3 0
VV -4.665394230640648
6 68 0 4 0
VV -4.665394230640648
6 68 1 0 0
VV -4.665394230623708
6 68 1 1 0
VV -4.665394229026281
6 68 1 2 0
VV -4.665394073927645
6 68 1 3 0
VV -4.665378862599415
6 68 1 4 0
VV -4.655931001615227
6 69 0 0 0
VV -4.590263014008508
6 69 0 1 0
VV -4.590263014008508
6 69 0 2 0
VV -

6 96 0 1 0
VV -2.9571626299872484
6 96 0 2 0
VV -2.9571626299872484
6 96 0 3 0
VV -2.9571626299872475
6 96 0 4 0
VV -2.9571626299872475
6 96 1 0 0
VV -2.9571626299872484
6 96 1 1 0
VV -2.9571626299872484
6 96 1 2 0
VV -2.9571626299872484
6 96 1 3 0
VV -2.9571626299872475
6 96 1 4 0
VV -2.9571626299872475
6 97 0 0 0
VV -2.909304276060011
6 97 0 1 0
VV -2.90930427606001
6 97 0 2 0
VV -2.909304276060011
6 97 0 3 0
VV -2.909304276060011
6 97 0 4 0
VV -2.909304276060011
6 97 1 0 0
VV -2.909304276060011
6 97 1 1 0
VV -2.90930427606001
6 97 1 2 0
VV -2.909304276060011
6 97 1 3 0
VV -2.909304276060011
6 97 1 4 0
VV -2.909304276060011
6 98 0 0 0
VV -2.862216511745271
6 98 0 1 0
VV -2.8622165117452703
6 98 0 2 0
VV -2.8622165117452703
6 98 0 3 0
VV -2.8622165117452707
6 98 0 4 0
VV -2.8622165117452703
6 98 1 0 0
VV -2.862216511745271
6 98 1 1 0
VV -2.8622165117452703
6 98 1 2 0
VV -2.8622165117452703
6 98 1 3 0
VV -2.8622165117452707
6 98 1 4 0
VV -2.8622165117452703
6 99 0 0 0
VV -2.81588718459

VV -10.780632629627279
5 23 0 3 0
VV -10.353991105114437
5 23 0 4 0
VV -9.872078976138575
5 23 1 0 0
VV -9.922027545524003
5 23 1 1 0
VV -9.405944479463882
5 23 1 2 0
VV -8.86860719840191
5 23 1 3 0
VV -8.323891755686132
5 23 1 4 0
VV -7.783404111302394
5 24 0 0 0
VV -11.24797551009441
5 24 0 1 0
VV -11.0117371087617
5 24 0 2 0
VV -10.681585995488547
5 24 0 3 0
VV -10.274793007812363
5 24 0 4 0
VV -9.809908651471988
5 24 1 0 0
VV -9.85844244494226
5 24 1 1 0
VV -9.356709215986779
5 24 1 2 0
VV -8.831001998491487
5 24 1 3 0
VV -8.295541543567586
5 24 1 4 0
VV -7.762176666190959
5 25 0 0 0
VV -11.10048672378363
5 25 0 1 0
VV -10.887851062481813
5 25 0 2 0
VV -10.579949739905214
5 25 0 3 0
VV -10.193152714196376
5 25 0 4 0
VV -9.745620942554813
5 25 1 0 0
VV -9.792691695040462
5 25 1 1 0
VV -9.305686801253527
5 25 1 2 0
VV -8.791947398642437
5 25 1 3 0
VV -8.266004845449945
5 25 1 4 0
VV -7.740013296139543
5 26 0 0 0
VV -10.950934524588378
5 26 0 1 0
VV -10.761357087158752
5 26 0 2 0
VV -

KeyboardInterrupt: 